In [25]:
!pip install git+https://github.com/airi-industrial-ai/fddbenchmark

  Cloning https://github.com/airi-industrial-ai/fddbenchmark to /private/var/folders/h7/3w3f6nt518qfc1cgwzz6x3z00000gn/T/pip-req-build-lh4ab0j_
  Running command git clone --filter=blob:none --quiet https://github.com/airi-industrial-ai/fddbenchmark /private/var/folders/h7/3w3f6nt518qfc1cgwzz6x3z00000gn/T/pip-req-build-lh4ab0j_
  Resolved https://github.com/airi-industrial-ai/fddbenchmark to commit 0d1b9c0f115835552b39fb9a72c6c319c35c767d
  Preparing metadata (setup.py) ... done


In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.preprocessing import StandardScaler
from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
import time
from tqdm.auto import tqdm, trange

from models import GNNTEP

Установка бенчмарка с датасетами:

pip install git+https://github.com/airi-industrial-ai/fddbenchmark

In [27]:
# загрузка датасета:
# small_tep - 18.9 МБ zip архив, 21 класс (20 неисправностей)
# rieth_tep - 1.84 ГБ zip архив, 21 класс (20 неисправностей)
# reinartz_tep - 1.88 ГБ zip фрхив, 29 классов (28 несиправностей)
dataset = FDDDataset(name='small_tep')
dataset.df.head()

Reading data/small_tep/dataset.csv:   0%|          | 0/153300 [00:00<?, ?it/s]

Reading data/small_tep/labels.csv:   0%|          | 0/153300 [00:00<?, ?it/s]

Reading data/small_tep/train_mask.csv:   0%|          | 0/153300 [00:00<?, ?it/s]

Reading data/small_tep/test_mask.csv:   0%|          | 0/153300 [00:00<?, ?it/s]

xmeas_1  xmeas_2  xmeas_3  xmeas_4  xmeas_5  xmeas_6  \
run_id    sample                                                         
413402073 1       0.25038   3674.0   4529.0   9.2320   26.889   42.402   
          2       0.25109   3659.4   4556.6   9.4264   26.721   42.576   
          3       0.25038   3660.3   4477.8   9.4426   26.875   42.070   
          4       0.24977   3661.3   4512.1   9.4776   26.758   42.063   
          5       0.29405   3679.0   4497.0   9.3381   26.889   42.650   

                  xmeas_7  xmeas_8  xmeas_9  xmeas_10  ...   xmv_2   xmv_3  \
run_id    sample                                       ...                   
413402073 1        2704.3   74.863   120.41   0.33818  ...  53.744  24.657   
          2        2705.0   75.000   120.41   0.33620  ...  53.414  24.588   
          3        2706.2   74.771   120.42   0.33563  ...  54.357  24.666   
          4        2707.2   75.224   120.39   0.33553  ...  53.946  24.725   
          5        2705.1   75.388   120.39   0.32632  ...  53.658  28.797   

                   xmv_4   xmv_5   xmv_6   xmv_7   xmv_8   xmv_9  xmv_10  \
run_id    sample                                                           
413402073 1       62.544  22.137  39.935  42.323  47.757  47.510  41.258   
          2       59.259  22.084  40.176  38.554  43.692  47.427  41.359   
          3       61.275  22.380  40.244  38.990  46.699  47.468  41.199   
          4       59.856  22.277  40.257  38.072  47.541  47.658  41.643   
          5       60.717  21.947  39.144  41.955  47.645  47.346  41.507   

                  xmv_11  
run_id    sample          
413402073 1       18.447  
          2       17.194  
          3       20.530  
          4       18.089  
          5       18.461  

[5 rows x 52 columns]

In [28]:
# стандартизация данных:
scaler = StandardScaler()
scaler.fit(dataset.df[dataset.train_mask])
dataset.df[:] = scaler.transform(dataset.df)

In [43]:
# даталоадер для обучения:
train_dl = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.train_mask,
    label=dataset.label,
    window_size=10,
    step_size=1,
    use_minibatches=True,
    batch_size=512,
    shuffle=True
)

# даталоадер для теста:
test_dl = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    label=dataset.label,
    window_size=10,
    step_size=1,
    use_minibatches=True,
    batch_size=512
)

# для измерения метрик:
evaluator = FDDEvaluator(step_size=1)

Creating sequence of samples: 100%|██████████| 105/105 [00:00<00:00, 3869.25it/s]


Параметры модели:

In [51]:
num_nodes = 52  # number of nodes
window_size = 10  # window size
ngnn = 1  # number of graph structure learning layers
gsllayer = 'constant'  # type of graph structure learning layer ('relu', 'directed', 'unidirected', 'undirected', 'constant')
nhidden = 256  # feature size in GCN layers
alpha = 0.1  # hyperparameter for weights of edges
k = None  # max number of edges for each node
out_channels = 21  # number of process states

Создание модели:

In [52]:
matrix = dict()
matrix[1] = [43]
matrix[2] = [42]
matrix[3] = [44]
matrix[4] = [45]
matrix[5] = [7, 20, 46]
matrix[6] = [7, 8, 9]
matrix[7] = [6, 45]
matrix[8] = [6]
matrix[9] = [9, 21, 46, 51]
matrix[10] = [47]
matrix[11] = [22, 48]
matrix[12] = [22, 48]
matrix[13] = [13, 22, 48]
matrix[14] = [48]
matrix[15] = [17, 48, 49, 50]
matrix[16] = [17, 45, 50]
matrix[18] = [17, 49, 50]
matrix[19] = []
matrix[20] = [5, 13]
matrix[21] = [9, 51]
matrix[22] = [11, 13, 48]
matrix[23] = [42, 43, 44]
matrix[24] = [42, 43, 44]
matrix[25] = [42, 43, 44, 46]
matrix[26] = [42, 43, 44, 46]
matrix[27] = [42, 43, 44, 46]
matrix[28] = [42, 43, 44]
matrix[29] = [47]
matrix[30] = [47]
matrix[31] = [47]
matrix[32] = [47]
matrix[33] = [46, 47]
matrix[34] = [46, 47]
matrix[35] = [46, 47]
matrix[36] = [47]
matrix[37] = [49]
matrix[38] = [49]
matrix[39] = [49]
matrix[40] = [49]
matrix[41] = [49]

# matrix[42] = [2, 5, 6, 23, 24, 25, 26, 27, 28]
# matrix[43] = [1, 5, 6, 23, 24, 25, 26, 27, 28]
# matrix[44] = [3, 5, 6, 23, 24, 25, 26, 27, 28]
# matrix[45] = [4, 7, 15, 16, 17]
# matrix[46] = [5, 9, 25, 26, 27, 33, 34, 35]
# matrix[47] = [10, 29, 30, 31, 32, 33, 34, 35, 36]
# matrix[48] = [11, 12, 13, 14, 15, 22]
# matrix[49] = [15, 17, 18, 37, 38, 39, 40, 41]
# matrix[50] = [15, 16, 17, 18, 52]
# matrix[51] = [9, 21]
# matrix[52] = [17, 50]

In [53]:
num_nodes = 52
print(num_nodes)

# Создать пустую матрицу smежности
adj_matrix = np.zeros((num_nodes, num_nodes), dtype=int)

# Заполнить матрицу на основе словаря
for node, neighbors in matrix.items():
    for neighbor in neighbors:
        # if neighbor >= 42:
        #     continue
        adj_matrix[node - 1, neighbor - 1] = 1  # -1,

52


In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GNNTEP(num_nodes, window_size, ngnn, gsllayer, nhidden,
               alpha, k, out_channels, device, adj_matrix)

Обучение:

In [55]:
model.to(device)
opt = Adam(model.parameters(), lr=0.001)
n_epochs = 10

weight = torch.ones(21) * 0.5
weight[1:] /= 20

model.train()
for e in range(n_epochs):
    av_loss = []
    for ts, _, label in tqdm(train_dl, desc='Steps ...', leave=False):
        label = torch.LongTensor(label).to(device)
        ts = torch.FloatTensor(ts).to(device)
        ts = torch.transpose(ts, 1, 2)
        logits = model(ts)
        loss = F.cross_entropy(logits, label, weight=weight.to(device))
        opt.zero_grad()
        loss.backward()
        opt.step()
        av_loss.append(loss.item())
    print(f'Epoch: {e + 1:2d}/{n_epochs}, average CE loss: {sum(av_loss) / len(av_loss):.4f}')

Steps ...:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch:  1/10, average CE loss: 1.6754


Steps ...:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch:  2/10, average CE loss: 1.2651


Steps ...:   0%|          | 0/101 [00:00<?, ?it/s]

...

Epoch:  8/10, average CE loss: 0.7284


Steps ...:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch:  9/10, average CE loss: 0.7086


Steps ...:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch: 10/10, average CE loss: 0.6907


Метрики:

In [56]:
preds = []
labels = []
model.eval()
for sample, index, label in test_dl:
    sample = torch.FloatTensor(sample).to(device)
    sample = torch.transpose(sample, 1, 2)
    with torch.no_grad():
        logits = model(sample)
    pred = logits.argmax(axis=1).cpu().numpy()
    preds.append(pd.Series(pred, index=index, name='pred'))
    labels.append(pd.Series(label, index=index, name='pred'))
preds = pd.concat(preds)
labels = pd.concat(labels)

metrics = evaluator.evaluate(labels, preds)
acc = metrics['confusion_matrix'].diagonal().sum() / metrics['confusion_matrix'].sum()
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.6782


In [57]:
metrics['detection']

{'TPR': 0.7099625, 'FPR': 0.011785444472425083, 'ADD': 40.27835051546392}